# Amazon Redshift (Provisioned)


### Cluster Configuration
- **Cluster**: wk08-redshift-cluster
- **Database**: dev
- **User**: rsadmin
- **Region**: us-west-2

In [ ]:
import boto3
import time
import json
import pandas as pd
from datetime import datetime

session = boto3.Session(region_name='us-west-2', profile_name='default')
client = session.client('redshift-data')

CLUSTER_ID = 'wk08-redshift-cluster'
DATABASE = 'dev'
DB_USER = 'rsadmin'
REGION = 'us-west-2'

In [2]:
def execute_and_wait(sql, cluster_id, database, db_user):
    response = client.execute_statement(
        ClusterIdentifier=cluster_id,
        Database=database,
        DbUser=db_user,
        Sql=sql
    )
    stmt_id = response['Id']
    print(f"Statement ID: {stmt_id}")

    start_time = time.time()
    while True:
        status = client.describe_statement(Id=stmt_id)
        if status['Status'] in ['FINISHED', 'FAILED', 'ABORTED']:
            break
        time.sleep(2) 
        
    end_time = time.time()
    wall_clock_time = end_time - start_time
    
    print(f"Status: {status['Status']}")
    if status['Status'] == 'FINISHED':
        duration_ns = status.get('Duration', 0)
        duration_sec = duration_ns / 1_000_000_000 if duration_ns > 0 else 0
        print(f"Execution time: {duration_sec:.2f} seconds")
        print(f"Wall clock time: {wall_clock_time:.2f} seconds")
        
    return status

In [4]:

# Example: Run EXPLAIN and capture plan
query_sql = """
SELECT 'optimized_orders' as table_name, COUNT(*) as row_count
FROM optimized_orders;
"""

explain_result = execute_and_wait(
    f"EXPLAIN {query_sql}",
    # grab the below values from your stack output
    cluster_id='wk08-redshift-cluster',
    database='dev',
    db_user='rsadmin'
)


Statement ID: 399a1dd5-b870-40b5-acd5-52517337b15f
Status: FINISHED
Execution time: 0.03 seconds
Wall clock time: 2.14 seconds
Status: FINISHED
Execution time: 0.03 seconds
Wall clock time: 2.14 seconds


In [9]:
def get_query_results(statement_id, max_rows=100):
    response = client.get_statement_result(
        Id=statement_id,
        NextToken=''
    )
    return response

In [10]:
s_id='399a1dd5-b870-40b5-acd5-52517337b15f'
get_query_results(s_id)

{'ColumnMetadata': [{'isCaseSensitive': True,
   'isCurrency': False,
   'isSigned': False,
   'label': 'QUERY PLAN',
   'length': 0,
   'name': 'QUERY PLAN',
   'nullable': 1,
   'precision': 65535,
   'scale': 0,
   'schemaName': '',
   'tableName': '',
   'typeName': 'varchar'}],
 'Records': [[{'stringValue': 'XN Aggregate  (cost=375000.00..375000.00 rows=1 width=0)'}],
  [{'stringValue': '  ->  XN Seq Scan on optimized_orders  (cost=0.00..300000.00 rows=30000000 width=0)'}]],
 'TotalNumRows': 2,
 'ResponseMetadata': {'RequestId': '46d66250-6fa3-461e-b13d-8a3ac52eac23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '46d66250-6fa3-461e-b13d-8a3ac52eac23',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '436',
   'date': 'Fri, 21 Nov 2025 03:05:52 GMT',
   'connection': 'close'},
  'RetryAttempts': 0}}

## Query Definitions

We'll execute two analytical queries against both the Spectrum external table and the native Redshift table to compare performance.

In [ ]:
# Query 1: Aggregation with selective time filter
query1_template = """
SELECT
    category,
    region,
    COUNT(*) as order_count,
    SUM(extended_price) as total_revenue,
    AVG(discount_rate) as avg_discount,
    COUNT(DISTINCT product_sku) as unique_products
FROM {table_name}
WHERE ts >= '2024-06-01' AND ts < '2024-07-01'
    AND category IN ('analytics', 'compute', 'observability')
    AND region IN ('us-east', 'us-west', 'eu-west')
GROUP BY category, region
ORDER BY total_revenue DESC;
"""

# Query 2: Multi-range temporal scan
query2_template = """
SELECT
    category,
    status,
    COUNT(*) as order_count,
    SUM(extended_price) as total_revenue,
    AVG(quantity) as avg_quantity,
    MIN(ts) as earliest_order,
    MAX(ts) as latest_order
FROM {table_name}
WHERE (
    (ts >= '2023-03-01' AND ts < '2023-03-15')
    OR (ts >= '2023-09-01' AND ts < '2023-09-15')
    OR (ts >= '2024-03-01' AND ts < '2024-03-15')
)
    AND category = 'analytics'
GROUP BY category, status
ORDER BY total_revenue DESC;
"""

# Tables
spectrum_table = "spectrum_raw.orders"
native_table = "optimized_orders"


## Performance Analysis Results

### Query 1: Aggregation with Selective Time Filter

This query tests performance on time-based filtering with aggregations across multiple dimensions.

In [44]:
# Dictionary to store results
results = {
    'query1': {'spectrum': {}, 'native': {}},
    'query2': {'spectrum': {}, 'native': {}}
}

print("Starting Query 1 Analysis - Spectrum Table")

# Query 1 - Spectrum EXPLAIN
query1_spectrum = query1_template.format(table_name=spectrum_table)
explain1_spectrum = execute_and_wait(
    f"EXPLAIN {query1_spectrum}",
    CLUSTER_ID, DATABASE, DB_USER
)
results['query1']['spectrum']['explain'] = explain1_spectrum

Starting Query 1 Analysis - Spectrum Table
Statement ID: 4deab60c-80d2-4411-a37b-d6fa45d76975
Status: FINISHED
Execution time: 0.30 seconds
Wall clock time: 2.15 seconds


In [14]:
# Query 1 - Spectrum execution
query1_spectrum_result = execute_and_wait(
    query1_spectrum,
    CLUSTER_ID, DATABASE, DB_USER
)
results['query1']['spectrum']['execution'] = query1_spectrum_result

if query1_spectrum_result['Status'] == 'FINISHED':
    spectrum_q1_duration = query1_spectrum_result.get('Duration', 0) / 1_000_000_000
    print(f"\nQuery 1 Spectrum completed in {spectrum_q1_duration:.2f} seconds")
else:
    print(f"\nQuery 1 Spectrum failed: {query1_spectrum_result.get('Error', 'Unknown error')}")

Statement ID: ff9df78f-be2a-4542-8c52-7ec21a6cc42d
Status: FINISHED
Execution time: 65.44 seconds
Wall clock time: 66.39 seconds

Query 1 Spectrum completed in 65.44 seconds
Status: FINISHED
Execution time: 65.44 seconds
Wall clock time: 66.39 seconds

Query 1 Spectrum completed in 65.44 seconds


In [15]:
print("Starting Query 1 Analysis - Native Table")

# Query 1 - Native EXPLAIN
query1_native = query1_template.format(table_name=native_table)
explain1_native = execute_and_wait(
    f"EXPLAIN {query1_native}",
    CLUSTER_ID, DATABASE, DB_USER
)
results['query1']['native']['explain'] = explain1_native

Starting Query 1 Analysis - Native Table
Statement ID: 104f9e27-e36e-4171-ae44-7e2278639841
Statement ID: 104f9e27-e36e-4171-ae44-7e2278639841
Status: FINISHED
Execution time: 0.04 seconds
Wall clock time: 2.13 seconds
Status: FINISHED
Execution time: 0.04 seconds
Wall clock time: 2.13 seconds


In [16]:
# Query 1 - Native execution
query1_native_result = execute_and_wait(
    query1_native,
    CLUSTER_ID, DATABASE, DB_USER
)
results['query1']['native']['execution'] = query1_native_result

if query1_native_result['Status'] == 'FINISHED':
    native_q1_duration = query1_native_result.get('Duration', 0) / 1_000_000_000
    print(f"\nQuery 1 Native completed in {native_q1_duration:.2f} seconds")
else:
    print(f"\nQuery 1 Native failed: {query1_native_result.get('Error', 'Unknown error')}")

Statement ID: 244ad6b0-f155-4d7c-97c0-c3691b1b65c1
Status: FINISHED
Execution time: 5.26 seconds
Wall clock time: 6.31 seconds

Query 1 Native completed in 5.26 seconds
Status: FINISHED
Execution time: 5.26 seconds
Wall clock time: 6.31 seconds

Query 1 Native completed in 5.26 seconds


### Query 2: Multi-range Temporal Scan

This query tests performance on complex temporal filtering with multiple date ranges.

In [17]:
print("Starting Query 2 Analysis - Spectrum Table")

# Query 2 - Spectrum EXPLAIN
query2_spectrum = query2_template.format(table_name=spectrum_table)
explain2_spectrum = execute_and_wait(
    f"EXPLAIN {query2_spectrum}",
    CLUSTER_ID, DATABASE, DB_USER
)
results['query2']['spectrum']['explain'] = explain2_spectrum

Starting Query 2 Analysis - Spectrum Table
Statement ID: e061e920-f25d-4a1a-bc11-aa3afa001106
Statement ID: e061e920-f25d-4a1a-bc11-aa3afa001106
Status: FINISHED
Execution time: 0.25 seconds
Wall clock time: 2.13 seconds
Status: FINISHED
Execution time: 0.25 seconds
Wall clock time: 2.13 seconds


In [18]:
# Query 2 - Spectrum execution
query2_spectrum_result = execute_and_wait(
    query2_spectrum,
    CLUSTER_ID, DATABASE, DB_USER
)
results['query2']['spectrum']['execution'] = query2_spectrum_result

if query2_spectrum_result['Status'] == 'FINISHED':
    spectrum_q2_duration = query2_spectrum_result.get('Duration', 0) / 1_000_000_000
    print(f"\nQuery 2 Spectrum completed in {spectrum_q2_duration:.2f} seconds")
else:
    print(f"\nQuery 2 Spectrum failed: {query2_spectrum_result.get('Error', 'Unknown error')}")

Statement ID: 0f731041-d8b0-4c18-ae4f-d7403b4b66ec
Status: FINISHED
Execution time: 63.98 seconds
Wall clock time: 64.45 seconds

Query 2 Spectrum completed in 63.98 seconds
Status: FINISHED
Execution time: 63.98 seconds
Wall clock time: 64.45 seconds

Query 2 Spectrum completed in 63.98 seconds


In [19]:
print("Starting Query 2 Analysis - Native Table")

# Query 2 - Native EXPLAIN
query2_native = query2_template.format(table_name=native_table)
explain2_native = execute_and_wait(
    f"EXPLAIN {query2_native}",
    CLUSTER_ID, DATABASE, DB_USER
)
results['query2']['native']['explain'] = explain2_native

Starting Query 2 Analysis - Native Table
Statement ID: a060ecb9-fae4-4fa7-959b-21a325c0241a
Statement ID: a060ecb9-fae4-4fa7-959b-21a325c0241a
Status: FINISHED
Execution time: 0.04 seconds
Wall clock time: 2.15 seconds
Status: FINISHED
Execution time: 0.04 seconds
Wall clock time: 2.15 seconds


In [20]:
# Query 2 - Native execution
query2_native_result = execute_and_wait(
    query2_native,
    CLUSTER_ID, DATABASE, DB_USER
)
results['query2']['native']['execution'] = query2_native_result

if query2_native_result['Status'] == 'FINISHED':
    native_q2_duration = query2_native_result.get('Duration', 0) / 1_000_000_000
    print(f"\nQuery 2 Native completed in {native_q2_duration:.2f} seconds")
else:
    print(f"\nQuery 2 Native failed: {query2_native_result.get('Error', 'Unknown error')}")

Statement ID: 4fd3d5ca-4a97-4179-8dc3-1d39b94f2a3c
Status: FINISHED
Execution time: 6.50 seconds
Wall clock time: 8.37 seconds

Query 2 Native completed in 6.50 seconds
Status: FINISHED
Execution time: 6.50 seconds
Wall clock time: 8.37 seconds

Query 2 Native completed in 6.50 seconds


## Table Metadata and Statistics

After running all queries, let's analyze the native table and get storage comparison metrics.

In [21]:
# Run ANALYZE on the native table
analyze_result = execute_and_wait(
    "ANALYZE optimized_orders;",
    CLUSTER_ID, DATABASE, DB_USER
)

Statement ID: cafc8b4f-7254-4211-b928-46a84ab96c49
Status: FINISHED
Execution time: 0.35 seconds
Wall clock time: 2.14 seconds
Status: FINISHED
Execution time: 0.35 seconds
Wall clock time: 2.14 seconds


In [22]:
# Get table metadata from svv_table_info
table_stats_sql = """
SELECT
    "table",
    size as size_mb,
    tbl_rows,
    sortkey1,
    diststyle,
    unsorted as unsorted_pct,
    encoded
FROM svv_table_info
WHERE "table" = 'optimized_orders';
"""

table_stats_result = execute_and_wait(
    table_stats_sql,
    CLUSTER_ID, DATABASE, DB_USER
)

# Get the results
if table_stats_result['Status'] == 'FINISHED':
    table_stats_data = get_query_results(table_stats_result['Id'])
    print("Table statistics retrieved successfully")
else:
    print("Failed to get table statistics")

Statement ID: 7b2d49d0-55d1-4cc4-bb03-7dd2f4a424ef
Status: FINISHED
Execution time: 6.27 seconds
Wall clock time: 8.45 seconds
Table statistics retrieved successfully
Status: FINISHED
Execution time: 6.27 seconds
Wall clock time: 8.45 seconds
Table statistics retrieved successfully


In [23]:
table_stats_result

{'ClusterIdentifier': 'wk08-redshift-cluster',
 'CreatedAt': datetime.datetime(2025, 11, 20, 19, 14, 12, 689000, tzinfo=tzlocal()),
 'Database': 'dev',
 'DbUser': 'rsadmin',
 'Duration': 6269925890,
 'HasResultSet': True,
 'Id': '7b2d49d0-55d1-4cc4-bb03-7dd2f4a424ef',
 'QueryString': '\nSELECT\n    "table",\n    size as size_mb,\n    tbl_rows,\n    sortkey1,\n    diststyle,\n    unsorted as unsorted_pct,\n    encoded\nFROM svv_table_info\nWHERE "table" = \'optimized_orders\';\n',
 'RedshiftPid': 1073955057,
 'RedshiftQueryId': 3946,
 'ResultFormat': 'json',
 'ResultRows': 1,
 'ResultSize': 75,
 'Status': 'FINISHED',
 'UpdatedAt': datetime.datetime(2025, 11, 20, 19, 14, 19, 449000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '3696b5e5-dd57-4c8b-9850-d40c4f1ade54',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3696b5e5-dd57-4c8b-9850-d40c4f1ade54',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '561',
   'date': 'Fri, 21 Nov 2025 03:14:2

In [24]:
# Get row counts for comparison
row_count_sql = """
SELECT 'spectrum_raw.orders' as table_name, COUNT(*) as row_count
FROM spectrum_raw.orders
UNION ALL
SELECT 'optimized_orders' as table_name, COUNT(*) as row_count
FROM optimized_orders;
"""

row_count_result = execute_and_wait(
    row_count_sql,
    CLUSTER_ID, DATABASE, DB_USER
)

Statement ID: c2113a5c-65cf-49be-b59f-bbe214a28d2b
Status: FINISHED
Execution time: 4.44 seconds
Wall clock time: 6.38 seconds
Status: FINISHED
Execution time: 4.44 seconds
Wall clock time: 6.38 seconds


## Performance Analysis Summary


### 1. EXPLAIN Plan Analysis


In [ ]:
def display_explain_plan(explain_result, query_name, table_type):
    print(f"\n{query_name} - {table_type} Table EXPLAIN Plan:")
    print("=" * 60)
    
    if explain_result['Status'] == 'FINISHED':
        plan_data = get_query_results(explain_result['Id'])
        if plan_data and 'Records' in plan_data:
            for record in plan_data['Records']:
                if record and len(record) > 0 and 'stringValue' in record[0]:
                    print(record[0]['stringValue'])
        else:
            print("No plan data available")
    else:
        print(f"EXPLAIN failed: {explain_result.get('Error', 'Unknown error')}")

display_explain_plan(results['query1']['spectrum']['explain'], "Query 1", "Spectrum")
display_explain_plan(results['query1']['native']['explain'], "Query 1", "Native")
display_explain_plan(results['query2']['spectrum']['explain'], "Query 2", "Spectrum")
display_explain_plan(results['query2']['native']['explain'], "Query 2", "Native")


Query 1 - Spectrum Table EXPLAIN Plan:
XN Merge  (cost=1000300000345.97..1000300000348.75 rows=1114 width=295)
  Merge Key: sum(extended_price)
  ->  XN Network  (cost=1000300000345.97..1000300000348.75 rows=1114 width=295)
        Send to leader
        ->  XN Sort  (cost=1000300000345.97..1000300000348.75 rows=1114 width=295)
              Sort Key: sum(extended_price)
              ->  XN HashAggregate  (cost=300000278.45..300000289.59 rows=1114 width=295)
                    ->  XN S3 Query Scan orders  (cost=0.00..300000111.38 rows=11138 width=295)
                          ->  S3 Seq Scan spectrum_raw.orders location:"s3://csed516-shared-resources-au2025-sharedclassbucket-mc0zal7gjcmu/class08/raw" format:ION  (cost=0.00..300000000.00 rows=11138 width=295)
                                Filter: (("ts" < '2024-07-01 00:00:00'::timestamp without time zone) AND ("ts" >= '2024-06-01 00:00:00'::timestamp without time zone) AND (((category)::text = 'analytics'::text) OR ((category)::t

### 2. Storage Comparison

In [46]:
# Storage Comparison Table
print("Storage Comparison:")
print("=" * 110)

storage_data = [
    ["S3 Spectrum", "JSON", "~3 GB", "S3 storage is larger due to JSON text format and metadata overhead"],
    ["Redshift Native", "Columnar", "2332 MB", "Redshift native storage is comparatively very small using columnar format"]
]

storage_df = pd.DataFrame(storage_data, columns=["Storage Location", "Format", "Size", "Notes"])
print(storage_df.to_string(index=False))


Storage Comparison:
Storage Location   Format    Size                                                                     Notes
     S3 Spectrum     JSON   ~3 GB        S3 storage is larger due to JSON text format and metadata overhead
 Redshift Native Columnar 2332 MB Redshift native storage is comparatively very small using columnar format


### 3. Execution Time Comparison

In [ ]:
# Create performance comparison table
performance_data = []

q1_spectrum_time = results['query1']['spectrum']['execution'].get('Duration', 0) / 1_000_000_000
q1_native_time = results['query1']['native']['execution'].get('Duration', 0) / 1_000_000_000
q2_spectrum_time = results['query2']['spectrum']['execution'].get('Duration', 0) / 1_000_000_000
q2_native_time = results['query2']['native']['execution'].get('Duration', 0) / 1_000_000_000

q1_speedup = q1_spectrum_time / q1_native_time if q1_native_time > 0 else 0
q2_speedup = q2_spectrum_time / q2_native_time if q2_native_time > 0 else 0

performance_data = [
    ['Query 1 (Selective Time Filter)', f"{q1_spectrum_time:.2f}s", f"{q1_native_time:.2f}s", f"{q1_speedup:.1f}x"],
    ['Query 2 (Multi-range Temporal)', f"{q2_spectrum_time:.2f}s", f"{q2_native_time:.2f}s", f"{q2_speedup:.1f}x"]
]

performance_df = pd.DataFrame(performance_data, 
                            columns=['Query', 'Spectrum Time ', 'Native Time', 'Speedup'])

print("Query Performance Comparison:")

print(performance_df.to_string(index=False))


Query Performance Comparison:
                          Query Spectrum Time  Native Time Speedup
Query 1 (Selective Time Filter)         65.44s       5.26s   12.5x
 Query 2 (Multi-range Temporal)         63.98s       6.50s    9.8x


## Part 5 - Export Aggregated Data with UNLOAD

In [45]:
# Document Storage Comparison
print("Document Storage Comparison:")
print("=" * 110)

storage_data = [
    ["S3 Spectrum", "JSON", "~3 GB", "30,000,000" ,"Original source data, text-based"],
    ["Redshift Native", "Columnar", "2332 MB", "30,000,000" , "From svv_table_info size column"],
    ["S3 Summary (Parquet)", "Parquet", "168.4 KB","~4,380", "Aggregated output, columnar + compression"]
]

storage_df = pd.DataFrame(storage_data, columns=["Storage Location", "Format", "Size", "Row Count", "Notes"])
print(storage_df.to_string(index=False))


Document Storage Comparison:
    Storage Location   Format     Size  Row Count                                     Notes
         S3 Spectrum     JSON    ~3 GB 30,000,000          Original source data, text-based
     Redshift Native Columnar  2332 MB 30,000,000           From svv_table_info size column
S3 Summary (Parquet)  Parquet 168.4 KB     ~4,380 Aggregated output, columnar + compression


### Analysis Questions:

**1. How does the summary Parquet size compare to the raw JSON source? What explains the dramatic reduction?**

There is a drastic reduction in the size. The original JSON storage weighed 2-3 GB whereas the Parquet was just in KBs. 


**2. What is the compression ratio from 30M rows to ~4,380 rows? What aggregation caused this?**

The compression ratio was 6849x. The aggregation of COUNT, SUM, and AVG around revenue_flag classification logic and GROUP BY caused this.


**3. How many partition directories were created? What does each represent?**

Three partition directories were created based on the revenue_flag classification:

1. revenue_flag=high
2. revenue_flag=normal
3. revenue_flag=low

Each partition represents a business-defined revenue performance tier that enables efficient anomaly detection and monitoring.


**4. How does partitioning by revenue_flag enable efficient downstream queries?**

Partitioning by revenue_flag enables partition pruning, where queries that filter by revenue performance only scan relevant directories. For example, a query looking for high-revenue anomalies (WHERE revenue_flag = 'high') would only scan the high/ directory, skipping approximately 66% of the data in normal and low partitions.


**5. What business value does this pre-aggregated dataset provide for dashboards and monitoring?**

This dataset provides significant business value through faster executive dashboards that can efficiently filter by performance tiers. It enables historical trend analysis for forecasting, category performance comparison across time periods, and consistent business logic through revenue_flag definitions. 

### UNLOAD Operation Documentation


- **Execution Duration**: [Insert duration from describe-statement response] seconds


- **Number of Parquet Files Created**: 12
- **Total Output Size**: 168.4 KiB

- **Partition Structure**
    - **revenue_flag=high**: Days exceeding $25,000,000 in daily revenue
    - **revenue_flag=normal**: Days with revenue between $700,000 and $25,000,000
    - **revenue_flag=low**: Days with revenue below $700,000

- **Row Count**: 4,380 rows

- **Storage Efficiency Analysis:**

| Storage Method | Format | Size | Row Count | Notes |
|----------------|--------|------|-----------|-------|
| S3 Spectrum (Source) | JSON | ~3 GB | 30,000,000 | Original raw transaction data |
| Redshift Native | Columnar | 2,332 MB | 30,000,000 | Compressed columnar storage |
| S3 Summary (UNLOAD) | Parquet | 168.4 KiB | 4,380 | Daily aggregated summaries |


## Reflection:



In this lab, I implemented a complete data workflow using Amazon Redshift that demonstrated modern data lake and warehouse integration patterns. The workflow began with Redshift Spectrum, which allowed me to query 30 million JSON records stored directly in S3 without loading them into Redshift. This external table approach provided immediate access to the raw data but came with performance limitations due to the text-based JSON format and network overhead of querying data across S3.
The next phase involved loading this data into a native Redshift table with optimized storage characteristics. By using columnar compression, sort keys, and distribution styles, I transformed the same 30 million rows from approximately 3GB of S3 storage to just 2332 MB of highly compressed native storage. The performance improvements were dramatic, with native queries executing significantly faster than their counterparts leading to 10x speedup on an approximate average.

The final component involved using UNLOAD to export aggregated daily summaries back to S3 as partitioned Parquet files. This process reduced 30 million individual transactions to approximately 4,380 daily summary records, achieving a compression ratio of 6849:1. The resulting Parquet files partitioned by revenue performance flags, created an optimized dataset for downstream analytics tools.


Architectural Trade-offs:

Spectrum excels when we need to query large amounts of infrequently accessed data (aka cold data) without the overhead of loading and maintaining it in Redshift. This approach works well for exploratory data analysis, archival queries, or when storage costs outweigh query performance requirements. However, Spectrum queries are inherently slower due to network latency and JSON parsing overhead.

Native Redshift tables should be chosen when query performance is critical or when data is accessed frequently (aka hot). The upfront cost of loading and storing data in Redshift pays back through faster query execution, less storage, and other advanced features.


UNLOAD and Data Interoperability:

The UNLOAD operation represents a crucial component of modern data architectures by enabling the S3 to Redshift to S3 roundtrip pattern. This workflow allows organizations to leverage Redshift's analytical capabilities while maintaining data in the more flexible and cost-effective S3 ecosystem. Common use cases include pre-computing expensive aggregations for dashboard consumption, creating optimized datasets, and sharing processed data with external systems.

Parquet format provides exceptional cross-tool interoperability, enabling the same dataset to be consumed by Athena or Spark. The columnar structure and built-in compression make Parquet faster for analytical workloads across different computing environments. The partitioning strategy we implemented using revenue flags enables efficient partition pruning, allowing downstream tools to scan only relevant data subsets.


Comparison to Previous Tools:

Compared to Athena from Week 3, Redshift Spectrum provides similar serverless S3 querying capabilities but with the added benefit of being integrated into a full data warehouse ecosystem. If I remember correctly from class, it was also mentioned that Redshift is better for IAM.

The decision between provisioned Redshift and serverless Athena depends primarily on usage patterns and performance requirements. Redshift becomes cost-effective when query volume is high and predictable, when sub-second response times are required, or when complex workload management is needed. Athena suits sporadic querying, cost-sensitive environments, and scenarios where infrastructure management overhead should be minimized. Polars from Week 2 serve as complementary tools in this ecosystem.


Production Considerations:

The decision to load data into Redshift versus keeping it in S3 are usually driven by access patterns, performance requirements, and cost considerations. Data that requires frequent access, complex joins, or superfast response times benefits from native Redshift storage. Archival/Cold data, infrequently accessed historical records, or datasets primarily used for batch processing are better suited for S3 storage with occasional Spectrum access.
Exporting optimized data back to S3 becomes valuable when creating curated datasets for specific use cases.
